In [23]:
from nlp import encoder, filereader
file = filereader("kanker_wiki_eng.txt")
encoded_data = encoder(file,1000,10)
encoded_list = encoded_data[0]
encoded_dict = encoded_data[1]
#encoded_dict
print(len(file))
print(len(encoded_list))

5555
5555


In [8]:
list_of_text = ["test.txt","kanker_wiki_eng.txt"]
def file_merger(list_of_files):
    list_of_words_per_file = []
    list_of_len_per_file = []
    final_list_of_words = []
    for file_index in range(len(list_of_files)):
        words = filereader(list_of_files[file_index])
        list_of_words_per_file.append(words)
        len_of_doc = len(words)
        list_of_len_per_file.append(len_of_doc)

    for list_index in range(len(list_of_words_per_file)):
        current_word_list = list_of_words_per_file[list_index]
        for word_index in range(len(current_word_list)):
            current_word = current_word_list[word_index]
            final_list_of_words.append(current_word)

    return final_list_of_words, list_of_len_per_file

test_merger = file_merger(list_of_text)
print(test_merger)
merged_list_of_words = test_merger[0]
merged_len_per_file = test_merger[1]

(['Er', 'ontstaan', 'gezwellen', '(tumoren).', 'Hoewel', 'het', 'woord', "'tumor'", 'voor', 'patiënten', 'vaak', 'een', 'angstige', 'bijklank', 'heeft', 'betekent', 'het', 'niet', 'meer', 'of', 'minder', 'dan', "'zwelling'.", 'Een', 'tumor', 'kan', 'zowel', 'goed-', 'als', 'kwaadaardig', 'zijn.', 'Een', 'goedaardige', 'tumor', 'wordt', 'ook', 'wel', 'benigne', 'genoemd,', 'een', 'kwaadaardige', 'maligne.', 'Bij', 'kanker', 'is', 'er', 'sprake', 'van', 'maligne', 'tumoren.', 'Kankerweefsel', 'geneest', 'niet', 'goed', 'en', 'gaat', 'makkelijk', 'bloeden.', 'Bloedverlies,', 'bijvoorbeeld', 'bij', 'ontlasting,', 'urine,', 'uit', 'de', 'tepel', 'of', 'bij', 'hoesten,', 'is', 'een', 'van', 'de', 'belangrijke', 'vroege', 'waarschuwingssymptomen.', 'De', 'gezwellen', 'drukken', 'op', 'andere', 'structuren', 'en', 'belemmeren', 'daarvan', 'de', 'werking.', 'Bij', 'de', 'darm', 'kan', 'bijvoorbeeld', 'passage', 'van', 'voedsel', 'onmogelijk', 'worden;', 'vanuit', '-', 'door', 'zwellingen', 'geb

In [24]:
max_tok = 1000
min_freqwi = 3

def group_encoder(max_tokens,min_freq,list_of_words,list_of_doc_len):
    uncoupled_token_lists_per_doc = []

    groupt_list_of_words, groupt_token_dict = encoder(list_of_words,max_tokens,min_freq)
    start = 0
    stop = 0
    # uncoupeling based on old len per doc
    for len_index in range(len(list_of_doc_len)):
        current_len = list_of_doc_len[len_index]-1
        stop = current_len + start
        words_in_file = groupt_list_of_words[start:stop]
        uncoupled_token_lists_per_doc.append(words_in_file)
        start = stop+1

    return uncoupled_token_lists_per_doc, groupt_token_dict

result_tokens_per_doc, token_dict = group_encoder(max_tok,min_freqwi,merged_list_of_words,merged_len_per_file)
#print(result_tokens_per_doc)

In [ ]:
import pandas as pd

def multi_hot_encoding(token_lists,tokens_dict,list_of_names):
    def word_checker(current_token_lists,key):

        result = False
        for word_index in range(len(current_token_lists)):
            current_word_in_tokens = current_token_lists[word_index]
            if key in current_word_in_tokens:
                result = True
            else:
                continue

        return result
    
    data = {}
    row_list = []
    
    for file_index in range(len(list_of_names)):
        file_name = list_of_names[file_index]
        data[file_name] = []

    for key in tokens_dict.keys():
        row_list.append(key)
        for file_index in range(len(list_of_names)):
            current_file = list_of_names[file_index]
            current_token_list = token_lists[file_index]
            bool_check = word_checker(current_token_list,key)
            if bool_check == True:
                old_list = data.get(current_file)
                old_list.append(1)
                data[current_file] = old_list
            else:
                old_list = data.get(current_file)
                old_list.append(0)
                data[current_file] = old_list

    df = pd.DataFrame(data, index=row_list)


    return df

x = multi_hot_encoding(result_tokens_per_doc,token_dict,list_of_text)
print(x)


      test.txt  kanker_wiki_eng.txt
1            1                    1
2            1                    1
3            1                    1
4            1                    1
5            1                    1
...        ...                  ...
996          0                    0
997          0                    1
998          0                    1
999          0                    1
1000         0                    1

[1000 rows x 2 columns]


In [36]:
def frequency_checker(token_lists,tokens_dict,list_of_names,result_type):
    def word_counter(current_token_lists,key):
        count = 0
        for word_index in range(len(current_token_lists)):
            current_word_in_tokens = current_token_lists[word_index]
            if key in current_word_in_tokens:
                count+=1
            else:
                continue
        return count
    

    
    data = {}
    total_token_counter = 0

    for file_index in range(len(list_of_names)):
        file_name = list_of_names[file_index]
        data[file_name] = []
    
    row_list = []
    for key in tokens_dict.keys():
        row_list.append(key)
        for file_index in range(len(list_of_names)):
            current_file = list_of_names[file_index]
            current_token_list = token_lists[file_index]
            word_count = word_counter(current_token_list,key)
            total_token_counter += word_count
            old_list = data.get(current_file)
            old_list.append(word_count)
            data[current_file] = old_list

    df = pd.DataFrame(data,index=row_list)
    if result_type == "frac":
        df = df / total_token_counter

    if result_type == "perc":
        df = df/total_token_counter*100

    return df

count_frame = frequency_checker(result_tokens_per_doc,token_dict,list_of_text,"perc")
print(count_frame)

      test.txt  kanker_wiki_eng.txt
1     0.027485             0.119102
2     0.027485             0.348145
3     0.054970             0.439762
4     0.119102             0.430600
5     0.091617             0.641319
...        ...                  ...
996   0.000000             0.000000
997   0.000000             0.036647
998   0.000000             0.036647
999   0.000000             0.036647
1000  0.000000             0.036647

[1000 rows x 2 columns]


In [ ]:

def tf_idf_calc(token_lists,tokens_dict,list_of_names):
    def word_counter(current_token_lists,key):
        count = 0
        for word_index in range(len(current_token_lists)):
            current_word_in_tokens = current_token_lists[word_index]
            if key in current_word_in_tokens:
                count+=1
            else:
                continue
        return count
    

    
    data = {}
    total_token_counter = []
    dict_list = []

    for file_index in range(len(list_of_names)):
        file_name = list_of_names[file_index]
        data[file_name] = []
        total_token_counter.append(0)
        empty_dict = {}
        dict_list.append(empty_dict)
    
    row_list = []
    for key in tokens_dict.keys():
        row_list.append(key)
        for file_index in range(len(list_of_names)):
            doc_count_dict = dict_list[file_index]
            current_file = list_of_names[file_index]
            current_token_list = token_lists[file_index]

            word_count = word_counter(current_token_list,key)
            doc_count_dict[key] = word_count
            dict_list.append(doc_count_dict)
            total_token_counter[file_index] += word_count

            old_list = data.get(current_file)
            old_list.append(word_count)
            data[current_file] = old_list

    df = pd.DataFrame(data,index=row_list)

    return df 

tf_idf_res = tf_idf_calc(result_tokens_per_doc,token_dict,list_of_text)
print(tf_idf_res)

{'test.txt': [3, 3, 6, 13, 10, 8, 8, 3, 15, 4, 2, 2, 5, 5, 4, 1, 7, 0, 1, 6, 2, 8, 3, 2, 1, 18, 3, 6, 0, 8, 5, 3, 2, 1, 0, 1, 4, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 2, 12, 0, 3, 1, 4, 25, 0, 4, 0, 0, 5, 0, 0, 4, 0, 6, 0, 2, 0, 2, 7, 3, 0, 4, 1, 9, 0, 0, 0, 0, 0, 9, 3, 1, 0, 0, 3, 1, 0, 1, 3, 0, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 1, 1, 1, 3, 0, 0, 1, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 1, 3, 1, 1, 0,